**Please set up your credentials JSON as GCP_CREDENTIALS secrets**

In [4]:
import os
import json

with open("gcs.json", "r") as f:
    credentials = json.load(f)

os.environ["DESTINATION__CREDENTIALS"] = json.dumps(credentials)
os.environ["BUCKET_URL"] = "gs://dezoomcamp_module4_2026"

In [5]:
import dlt
import requests
import pandas as pd
from dlt.destinations import filesystem
from io import BytesIO

Ingesting parquet files to GCS.

In [10]:
# Define a dlt source to download and process Parquet files as resources
@dlt.source(name="rides")
def download_parquet():
    prefix = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata"
    for month in range(1, 13):
        month_str = f"{month:02d}"
        file_name = f"yellow_tripdata_2020-{month_str}.parquet"
        url = f"{prefix}_2020-{month_str}.parquet"
        response = requests.get(url)

        df = pd.read_parquet(BytesIO(response.content))

        # Return the dataframe as a dlt resource for ingestion
        yield dlt.resource(df, name=file_name)


# Initialize the pipeline
pipeline = dlt.pipeline(
    pipeline_name="rides_pipeline",
    destination=filesystem(layout="{schema_name}/{table_name}.{ext}"),
    dataset_name="rides_dataset",
)

load_info = pipeline.run(download_parquet(), loader_file_format="parquet")

# Print the results
print(load_info)


2026-02-09 09:54:49,516|[WARNING]|61267|8600576192|dlt|validate.py|verify_normalized_table:91|In schema `rides`: The following columns in table 'yellow_tripdata_2020_05_parquet' did not receive any data during this load and therefore could not have their types inferred:
  - airport_fee

Unless type hints are provided, these columns will not be materialized in the destination.
One way to provide type hints is to use the 'columns' argument in the '@dlt.resource' decorator.  For example:

@dlt.resource(columns={'airport_fee': {'data_type': 'text'}})

2026-02-09 09:54:49,517|[WARNING]|61267|8600576192|dlt|validate.py|verify_normalized_table:91|In schema `rides`: The following columns in table 'yellow_tripdata_2020_04_parquet' did not receive any data during this load and therefore could not have their types inferred:
  - airport_fee

Unless type hints are provided, these columns will not be materialized in the destination.
One way to provide type hints is to use the 'columns' argument in t

Pipeline rides_pipeline load step completed in 2 minutes and 32.44 seconds
1 load package(s) were loaded to destination filesystem and into dataset rides_dataset
The filesystem destination used gs://dezoomcamp_module4_2026 location to store data
Load package 1770584083.236244 is LOADED and contains no failed jobs


Ingesting data to Database

In [ ]:
# Define a dlt resource to download and process Parquet files as single table
@dlt.resource(name="rides", write_disposition="replace")
def download_parquet():
    prefix = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata'

    for month in range(1, 13):
        month_str = f"{month:02d}"
        url = f"{prefix}_2019-{month_str}.parquet"
        response = requests.get(url)

        df = pd.read_parquet(BytesIO(response.content))

        yield df


# Initialize the pipeline
pipeline = dlt.pipeline(
    pipeline_name="rides_pipeline",
    destination="duckdb",  # Use DuckDB for testing
    # destination="bigquery",  # Use BigQuery for production
    dataset_name="rides_dataset",
)

# Run the pipeline to load Parquet data into DuckDB
info = pipeline.run(download_parquet)

# Print the results
print(info)


In [ ]:
import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# Set search path to the dataset
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")

# Describe the dataset to see loaded tables
res = conn.sql("DESCRIBE").df()
print(res)

In [ ]:
# provide a resource name to query a table of that name
with pipeline.sql_client() as client:
    with client.execute_query(f"SELECT count(1) FROM rides") as cursor:
        data = cursor.df()
print(data)